In [3]:
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv

from core_ds4a_project import cleaning, columns as project_columns, datasets

%load_ext autoreload
%autoreload 1
%aimport core_ds4a_project, core_ds4a_project.cleaning, core_ds4a_project.columns, core_ds4a_project.datasets

pd.set_option("display.max_columns", None)

Environment variables:

In [4]:
load_dotenv('envvars')

ROOT_DATA_PATH = os.environ.get('ROOT_DATA_PATH')
RAW_DATA_PATH = os.environ.get('RAW_DATA_PATH') or f'{ROOT_DATA_PATH}/raw'

Reading data:

In [5]:
(cartera_df, clientes_df, colocacion_df) = datasets.read_joining_datasets(dir_path=RAW_DATA_PATH)

Draft:

In [6]:
clientes_df[['ACTIVIDAD', 'OCUPACION', 'OFICIO', 'PROFESION', 'ACTIVIDAD_CIIU_PRIMARIA', 'ACTIVIDAD_ECONOMICA']]

,ACTIVIDAD,OCUPACION,OFICIO,PROFESION,ACTIVIDAD_CIIU_PRIMARIA,ACTIVIDAD_ECONOMICA
0,Comercio |C,Independiente |2,NaN,NaN,NaN,Negocio o Microempresa |NM
1,Produccion |P,Independiente |2,GANADERIA,GANADERIA,NaN,Negocio o Microempresa |NM
2,Comercio |C,Independiente |2,NaN,ADMON DE NEGOCIOS,NaN,Negocio o Microempresa |NM
3,Produccion |P,Independiente |2,GANADERIA,GANADERIA,CRÍA DE GANADO BOVINO Y BUFALINO |141|A,Negocio o Microempresa |NM
4,Comercio |C,Independiente |2,ALMACEN Y TALLER DE MOTOS,ADMON DE EMPRESAS,NaN,Negocio o Microempresa |NM
...,...,...,...,...,...,...
22960,Servicios |S,Independiente |2,ARRENDAMIENTOS,ADMON DE NEGOCIOS,ALQUILER Y ARRENDAMIENTO DE OTROS EFECTOS PERS...,Negocio o Microempresa |NM
22961,Produccion |P,Independiente |2,GANADERIA CRIA Y CEBA Y CULTIV,AGRICULTURA,NaN,Negocio o Microempresa |NM
22962,Servicios |S,Independiente |2,TAXISTA,NaN,TRANSPORTE DE CARGA POR CARRETERA |4923|H,Negocio o Microempresa |NM
22963,Servicios |S,Independiente |2,SALON DE BELLEZA,NaN,PELUQUERÍA Y OTROS TRATAMIENTOS DE BELLEZA |96...,Negocio o Microempresa |NM


In [7]:
clientes_df['ACTIVIDAD'].value_counts(dropna=False)

Produccion |P    10133
Comercio |C       7762
Servicios |S      5070
Name: ACTIVIDAD, dtype: int64

In [8]:
ss = clientes_df['OFICIO'].value_counts()

In [9]:
ss[ss > 100]

GANADERIA                     1874
AGRICULTOR                     979
COMERCIANTE                    706
AGRICULTURA                    654
TIENDA                         582
ARRENDADOR                     357
ARRIENDOS                      241
AVICOLA                        223
VENTA POR CATALOGO             213
PORCICULTURA                   190
VENTA DE MERCANCIA             189
014 Ganadería.                 179
INDEPENDIENTE                  157
VENTA DE BEBIDAS               154
RESTAURANTE                    144
AGROPECUARIO                   143
TRANSPORTES                    140
SALON DE BELLEZA               137
0113 Cultivo de hortalizas     137
VENTA DE COMIDAS RAPIDAS       120
AVICULTURA                     117
GANADERO                       114
ARRENDAMIENTOS                 101
Name: OFICIO, dtype: int64

Adding worst and last calification:

In [10]:
calificacion_by_obligacion = cartera_df.groupby(by=['CLIENTE'])['CALIFICACION_CIERRE']

calificacion_worst = calificacion_by_obligacion.max().rename('CALIFICACION_PEOR').reset_index()
calificacion_best = calificacion_by_obligacion.min()

cartera_df = cartera_df.sort_values(by='FECHA_CIERRE')
calificacion_last = (
    cartera_df[['CLIENTE', 'FECHA_CIERRE', 'CALIFICACION_CIERRE']]
    .drop_duplicates(keep='last', subset=['CLIENTE'])
    .drop(columns=['FECHA_CIERRE'])
    .rename(columns={'CALIFICACION_CIERRE': 'CALIFICACION_ULTIMA'})
)

clientes_df = (
    clientes_df
    .merge(calificacion_worst, on='CLIENTE')
    .merge(calificacion_last, on='CLIENTE')
)

clientes_df[['CLIENTE', 'CALIFICACION_PEOR', 'CALIFICACION_ULTIMA']].head()

,CLIENTE,CALIFICACION_PEOR,CALIFICACION_ULTIMA
0,FA4940,A,A
1,FA1852,A,A
2,FA2274,A,A
3,FA22995,A,A
4,FA15635,A,A


In [11]:
ind = (clientes_df['CALIFICACION_PEOR'] == clientes_df['CALIFICACION_ULTIMA'])

ind.sum(), (~ind).sum()

(21364, 1601)

In [12]:
clientes_df.loc[~ind, ['CLIENTE', 'CALIFICACION_PEOR', 'CALIFICACION_ULTIMA']]

,CLIENTE,CALIFICACION_PEOR,CALIFICACION_ULTIMA
16,FA4476,E,A
38,FA11312,D,C
52,FA23510,C,B
63,FA5451,B,A
64,FA10073,C,B
...,...,...,...
20789,FA5686,B,A
20793,FA5662,B,A
20942,FA7654,B,A
20977,FA4879,B,A


## Draft

In [13]:
cliente_obligacion_df = (
    cartera_df
    [['CLIENTE', 'OBLIGACION']]
    .drop_duplicates()
)

In [14]:
indices = cliente_obligacion_df.groupby('CLIENTE').size().sort_values()
indices

CLIENTE
FA1         1
FA23412     1
FA23414     1
FA23415     1
FA23416     1
           ..
FA18127    10
FA11827    10
FA19705    10
FA21616    11
FA767      11
Length: 22965, dtype: int64

### CLIENTE == "FA10006"

In [15]:
cliente_obligacion_df.query('CLIENTE == "FA10006"')

,CLIENTE,OBLIGACION
353597,FA10006,172000536


In [17]:
# colocacion_df.query('OBLIGACION == 172000536')

In [20]:
colocacion_df.query('OBLIGACION == 172000536')['OBSERVACIONES'].iloc[0]

'CLIENTE ANTIGUO. CON HISTORIAL DE CRÉDITO ENDEUDAMIENTO POR $11.000.000, COMPORTAMIENTO NORMAL, SE DEDICA A PRESTAR SUS SERVICIOS PROFESIONALES DE FOTOGRAFIA Y GRABACIÓN.SOLICITA FINANCIACIÓN PARA COM'

In [21]:
# cartera_df.query('OBLIGACION == 172000536').sort_values('FECHA_CIERRE')

### CLIENTE == "FA21616"

In [22]:
# clientes_df.query('CLIENTE == "FA21616"')

In [23]:
obligaciones = cliente_obligacion_df.query('CLIENTE == "FA21616"')['OBLIGACION']
obligaciones

725       179000113
155453    179000562
754064    179001193
650614    179001411
514685    199001682
657591    199002524
465214    209003025
779099    209003225
191733    209003512
539457    219000162
478629    219000396
Name: OBLIGACION, dtype: int64

In [24]:
# colocacion_df.query('OBLIGACION.isin(@obligaciones)').sort_values(by='FECHA_SOLICITUD')